In [12]:
# Mountain Car- v0 solution using action-value neural network function approximator

In [13]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.autograd import Variable

# One hidden layer function approximator
class action_value_function(nn.Module):

    def __init__(self,n_observations, n_actions):

        super(action_value_function, self).__init__()
        self.block = nn.Sequential(
            torch.nn.Linear(n_observations, 100, bias = False),
            torch.nn.Linear(100, n_actions, bias = False),
        )

    def forward(self, input):
        return self.block(input)
    
# I tried different weight initializations but found they did not perform well.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight, 0, 1)


In [14]:
import gym
import numpy as np

done = False
LEARNING_RATE = 0.1
DISCOUNT = 1.0
EPISODES = 25000
SHOW_EVERY = 300
BATCH_SIZE = 4
rewards_path = 'rewards.npy'


env = gym.make("CartPole-v0")
env.reset()

# Instaciate action_value_function class
q_hat = action_value_function(env.observation_space.shape[0], env.action_space.n)
q_hat_target = action_value_function(env.observation_space.shape[0], env.action_space.n)

In [15]:
# Test number of actions and other parameters
print('observation_space:',env.observation_space.low,'to',env.observation_space.high,'| Number of action values:',env.action_space.n)

observation_space: [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38] to [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38] | Number of action values: 2


In [16]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(q_hat.parameters(), lr=1e-5)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
env.observation_space.low

array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32)

In [17]:
env.close()
# cart_position, cart_velocity, angle, pole_tip_velocity = env.reset()
print(env.reset())
# print('cart_position: %5.2f | cart_velocity: %5.2f | angle: %5.2f | Pole Tip velocity: %5.2f'%(cart_position, cart_velocity, angle, pole_tip_velocity))

[-0.01279914 -0.01352579  0.01693123  0.00296156]


In [18]:
# # Dynamic plots of rewards
# import matplotlib.pyplot as plt
# import matplotlib.animation as animation
# import time


# %matplotlib qt

# fig = plt.figure()
# ax1 = fig.add_subplot(1,1,1)

# def animate(i):
# #     pullData = open("sampleText.txt","r").read()
# #     dataArray = pullData.split('\n')
# #     xar = []
# #     yar = []
# #     for eachLine in dataArray:
# #         if len(eachLine)>1:
# #             x,y = eachLine.split(',')
# #             xar.append(int(x))
# #             yar.append(int(y))
#     rewards = np.load(rewards_path)
#     ax1.clear()
#     ax1.plot(rewards)
# ani = animation.FuncAnimation(fig, animate, interval=1000)
# plt.show()

In [21]:
avg_loss = 0
epsilon = 0.9
curr_high_pos = 0
curr_low_pos = 0
successes = 0
UPD = 50
upd = 0
STEPS = 200
ep_rewards = np.array([])
q_hat_target.load_state_dict(q_hat.state_dict())

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [23]:
# Train agent
import random

for episode in range(EPISODES):
   
    state = torch.from_numpy(env.reset()).float()
    done = False
    ep_reward = 0
    
    if episode%SHOW_EVERY == 0:
        print('-'*70)
        RENDER = True
    else:
        RENDER = False

    running_loss = 0
    cont = 0

#     for step in range(STEPS):
    while not done:
        if RENDER:
            env.render()
        upd += 1;
        if upd%UPD == 0:
            q_hat_target.load_state_dict(q_hat.state_dict())
            upd = 0
            
        q = q_hat(Variable(state))
        
        if random.random() < epsilon:
            action = env.action_space.sample()
        else:
            action = torch.argmax(q).item()
        
        next_state, reward, done, _ = env.step(action)
        next_state = torch.from_numpy(next_state).float()
        
                    
#         print(count)
             
#         if next_state[0] >= 2.4 or next_state[0] <= -2.4:
#             reward -= .1
#         if next_state[2] >= 12 or next_state[2] < -12:
#             reward -= 1
            
        q_target = q.clone()
#         q_target = Variable(q_target.data)
        
        q_target[action] = torch.tensor(reward) + DISCOUNT * torch.max(q_hat_target(next_state))
        if done:
            q_target[action] = torch.tensor(reward)
            
            
        # Calculate loss
        loss = criterion(q, q_target.detach())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        state = next_state
        
        ep_reward += reward
        running_loss += loss.item()
        
    epsilon *= .99999
#     scheduler.step()
    ep_rewards = np.append(ep_rewards, ep_reward)

    avg_loss += running_loss
    env.close()
    if episode%100 == 0:
        print('Episode: %5d | Average Loss: %5.2f | Epsilon: %4.2f | Avg. Reward: %5.2f'%(episode, avg_loss/100, epsilon,np.mean(ep_rewards[-100:])))
        avg_loss = 0
        ep_reward = 00
        np.save(rewards_path, ep_rewards)

# env.close()

----------------------------------------------------------------------
Episode:     0 | Average Loss: 11.53 | Epsilon: 0.70 | Avg. Reward: 14.06
Episode:   100 | Average Loss: 12.04 | Epsilon: 0.70 | Avg. Reward: 14.55
Episode:   200 | Average Loss: 12.04 | Epsilon: 0.70 | Avg. Reward: 14.58
----------------------------------------------------------------------
Episode:   300 | Average Loss: 12.08 | Epsilon: 0.70 | Avg. Reward: 14.49
Episode:   400 | Average Loss: 13.00 | Epsilon: 0.70 | Avg. Reward: 15.80
Episode:   500 | Average Loss: 12.64 | Epsilon: 0.70 | Avg. Reward: 15.40
----------------------------------------------------------------------
Episode:   600 | Average Loss: 11.65 | Epsilon: 0.70 | Avg. Reward: 14.21
Episode:   700 | Average Loss: 12.59 | Epsilon: 0.70 | Avg. Reward: 14.96
Episode:   800 | Average Loss: 13.07 | Epsilon: 0.70 | Avg. Reward: 15.55
----------------------------------------------------------------------
Episode:   900 | Average Loss: 11.83 | Epsilon: 0.

----------------------------------------------------------------------
Episode:  8400 | Average Loss: 12.10 | Epsilon: 0.64 | Avg. Reward: 13.42
Episode:  8500 | Average Loss: 12.08 | Epsilon: 0.64 | Avg. Reward: 13.44
Episode:  8600 | Average Loss: 12.29 | Epsilon: 0.64 | Avg. Reward: 14.03
----------------------------------------------------------------------
Episode:  8700 | Average Loss: 12.64 | Epsilon: 0.64 | Avg. Reward: 13.79
Episode:  8800 | Average Loss: 12.38 | Epsilon: 0.64 | Avg. Reward: 13.41
Episode:  8900 | Average Loss: 12.89 | Epsilon: 0.64 | Avg. Reward: 14.03
----------------------------------------------------------------------
Episode:  9000 | Average Loss: 12.71 | Epsilon: 0.64 | Avg. Reward: 13.98
Episode:  9100 | Average Loss: 12.42 | Epsilon: 0.64 | Avg. Reward: 13.42
Episode:  9200 | Average Loss: 12.03 | Epsilon: 0.64 | Avg. Reward: 13.23
----------------------------------------------------------------------
Episode:  9300 | Average Loss: 12.19 | Epsilon: 0.

----------------------------------------------------------------------
Episode: 16800 | Average Loss: 13.32 | Epsilon: 0.59 | Avg. Reward: 13.46
Episode: 16900 | Average Loss: 13.49 | Epsilon: 0.59 | Avg. Reward: 13.68
Episode: 17000 | Average Loss: 13.11 | Epsilon: 0.59 | Avg. Reward: 13.12
----------------------------------------------------------------------
Episode: 17100 | Average Loss: 13.01 | Epsilon: 0.59 | Avg. Reward: 13.03
Episode: 17200 | Average Loss: 13.82 | Epsilon: 0.59 | Avg. Reward: 13.71
Episode: 17300 | Average Loss: 14.06 | Epsilon: 0.59 | Avg. Reward: 13.70
----------------------------------------------------------------------
Episode: 17400 | Average Loss: 12.99 | Epsilon: 0.59 | Avg. Reward: 13.07
Episode: 17500 | Average Loss: 13.56 | Epsilon: 0.59 | Avg. Reward: 13.57
Episode: 17600 | Average Loss: 13.49 | Epsilon: 0.59 | Avg. Reward: 13.53
----------------------------------------------------------------------
Episode: 17700 | Average Loss: 12.66 | Epsilon: 0.

In [ ]:
# # Save model
# torch.save(q_hat.state_dict(),'./models/cartpole-v0.pth')
# # q = action_value_function()
# q.load_state_dict(torch.load('./models/trained/cartpole-v0.pth'))

In [ ]:
# Test your policy
env.close()
state = torch.from_numpy(env.reset()).float()
done = False 
#             pri
EPS_TEST = 0.05
tot_reward = 0
for step in range(200):
    env.render()
    q = q_hat((state))
    action = torch.argmax(q).item()
#     if random.random() < EPS_TEST:
#         action = env.action_space.sample() 
    next_state, reward, done, _ = env.step(action)
    tot_reward += reward
    next_state = torch.from_numpy(next_state).float()
    state = next_state
#     if reward == 1:
#         print('Hell Yes!')
        
print(tot_reward)
env.close()

In [ ]:
q_hat_target.load_state_dict('./models/cartpole-v0.pth')

In [ ]:
q_target